In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Definindo a função para avaliar os modelos
def model_evaluation(test, prediction):
    print('Matriz de Confusão\n', metrics.confusion_matrix(test, prediction)) 
    print(f'\nAcurácia\n {np.round(metrics.accuracy_score(test, prediction), 2)}%') 
    print('\nAcurácia Balanceada por classe\n', metrics.balanced_accuracy_score(test, prediction)) 
    print('\nPrecision\n', metrics.precision_score(test, prediction)) 
    print('\nRecall\n', metrics.recall_score(test, prediction)) 
    print('\nF1\n', metrics.f1_score(test, prediction))
    print('\nClassification Report\n', metrics.classification_report(test, prediction))

# Previsão de Insuficiência Cardíaca
## 12 características clínicas por prever eventos de morte.
----

### Sobre o Conjunto de Dados

*As doenças cardiovasculares (DCVs) são a **causa número 1** de morte em todo o mundo, levando cerca de **17,9 milhões de vidas** a cada ano, o que representa 31% de todas as mortes em todo o mundo.*

A insuficiência cardíaca é um evento comum causado por DCVs e este 
conjunto de dados contém 12 recursos que podem ser usados 
para prever a mortalidade por insuficiência cardíaca.


A maioria das doenças cardiovasculares pode ser prevenida abordando 
os fatores de risco comportamentais, como uso de tabaco, 
dieta não saudável e obesidade, sedentarismo e uso nocivo de álcool, 
usando estratégias para toda a população.


Pessoas com doenças cardiovasculares ou que apresentam alto risco cardiovascular 
(devido à presença de um ou mais fatores de risco, como hipertensão, 
diabetes, hiperlipidemia ou doença já estabelecida) precisam de detecção e 
gerenciamento precoces , em que um modelo de aprendizado de máquina pode ser de grande ajuda.*

### **Dicionário de dados**
* ***age*** - Idade
* ***anaemia*** - Diminuição de glóbulos vermelhos ou hemoglobina (booleano)
* ***creatinine_phosphokinase*** - Nível da enzima CPK no sangue (mcg / L)
* ***diabetes*** - Se o paciente tem diabetes (booleano)
* ***ejection_fraction*** - Porcentagem de sangue saindo do coração a cada contração (porcentagem)
* ***high_blood_pressure*** - Se o paciente tem hipertensão (booleano)
* ***platelets*** - Plaquetas no sangue (quiloplacas / mL)
* ***serum_creatinine*** - Nível de creatinina sérica no sangue (mg / dL)
* ***serum_sodium*** - Nível de sódio sérico no sangue (mEq / L)
* **smoking**** - Se o paciente fuma ou não (booleano)
* ***sex*** - Mulher ou homem (binário)
* ***time*** - Período de acompanhamento (dias)
* ***DEATH_EVENT*** - Se o paciente faleceu durante o período de acompanhamento (booleano)

# Explorando os dados
-----

In [ ]:
# Carrega o arquivo csv como DataFrame
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
# Exibe as 10 primeiras linhas
df.head(10)

In [ ]:
# Exibe algumas informações sobre as colunas do DataFrame
df.info()

In [ ]:
# Exibe informações estatísticas sobre o conjunto de dados
df.describe()

In [ ]:
# Exibe um mapa de calor de correlação entre variáveis
plt.figure(figsize=(15,8))
corr = df.corr()
sns.heatmap(corr, annot=True, vmin=-1, vmax=1);

In [ ]:
plt.figure(figsize=(9,8))
sns.set_theme(style="darkgrid")
sns.barplot(data=df[['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']])

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(x='DEATH_EVENT', data=df, hue='DEATH_EVENT');

In [ ]:
sns.displot(x='age', kind='kde',hue='sex',data=df, multiple="stack", fill=True)

In [ ]:
sns.pairplot(df.drop(['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking'], axis=1), hue='DEATH_EVENT', corner=True);

In [ ]:
df.columns

In [ ]:
#plt.figure(figsize=(10,8))
sns.countplot(x='smoking', data=df , hue='DEATH_EVENT');

# **Pré Processamento**

In [ ]:
# Salva o nome das colunas em uma lista
columns = list(df.columns)

# Labels
y = df.iloc[:,-1]

# One-hot Encoding
# X = pd.get_dummies(df, columns=['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking'])

# Guarda os valores do dataset sem a coluna de rótulos (variável alvo)
X = np.array(df.drop('DEATH_EVENT', axis=1))

print(columns)
print(X.shape)
print(y.shape)

In [ ]:
# Normalização dos dados
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Redução de dimensionalidade
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
X = pca.fit_transform(X)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
rskf = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=42)
for train_index, test_index in rskf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Random Forest Classifiers**

In [ ]:
# importar o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier
# Treinando o modelo do RF
rf = RandomForestClassifier(
    bootstrap=True,
    min_impurity_decrease=0.02, 
    random_state=42)
rf.fit(X_train, y_train)
# Realizando as primeiras previsões
y_predict = rf.predict(X_test)

In [ ]:
# Avaliando o resultado do modelo do RF
model_evaluation(test=y_test, prediction=y_predict)

# **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    C=0.1, 
    random_state=42).fit(X_train, y_train)
y_predict = lr.predict(X_test)

In [ ]:
matrix = metrics.confusion_matrix(y_test, y_predict)
sns.heatmap(matrix, annot=True, cmap="YlGnBu");

In [ ]:
# Avaliando o resultado do modelo de Regressão Logística
model_evaluation(test=y_test, prediction=y_predict)

# **SVC**

In [ ]:
from sklearn.svm import SVC
svc = SVC(
    kernel='sigmoid',
    random_state=42)
svc.fit(X_train, y_train)
y_predict = svc.predict(X_test)

In [ ]:
# Avaliando o resultado do modelo do SVM
print('Matriz de Confusão\n', metrics.confusion_matrix(y_test, y_predict)) 
print(f'\nAcurácia\n {np.round(metrics.accuracy_score(y_test, y_predict), 2)}%')  
print('\nAcurácia Balanceada por classe\n', metrics.balanced_accuracy_score(y_test, y_predict)) 
print('\nPrecision\n', metrics.precision_score(y_test, y_predict)) 
print('\nRecall\n', metrics.recall_score(y_test, y_predict)) 
print('\nF1\n', metrics.f1_score(y_test, y_predict))

# **SGD**

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(
    loss='hinge',
    penalty='l1',
    alpha=0.01,
    random_state=42)
sgd.fit(X_train, y_train)
y_predict = sgd.predict(X_test)

In [ ]:
model_evaluation(prediction=y_predict, test=y_test)

# **AdaBoost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
lrc = LogisticRegression(C=0.7)
dtc = DecisionTreeClassifier()
abc = AdaBoostClassifier(    
    base_estimator=lrc,
    random_state=42
).fit(X_train, y_train)

y_pred = abc.predict(X_test)

In [ ]:
model_evaluation(y_test, y_pred)